# load

In [1]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model2 = CNNPointDetector()
# 初始化模型
model2 = CNNPointDetector()
model2.load_state_dict(torch.load('fan_corner2_detector_1201.pth'))  # 加载模型参数
model2.train()

C:\Users\user\AppData\Local\Temp\ipykernel_15324\2895312645.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('fan_corner2_detector_1201

CNNPointDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

# newstart

In [2]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points2_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model2
model2 = CNNPointDetector()

In [5]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

# Training loop
num_epochs = 720
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model2(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min   or %-.3f hrs left'% ((num_epochs-epoch)/60))

Epoch [1/720], Loss: 5622519.6000 about 720.0 min   or 12.000 hrs left
Epoch [2/720], Loss: 5601296.5000 about 719.0 min   or 11.983 hrs left
Epoch [3/720], Loss: 5518765.6000 about 718.0 min   or 11.967 hrs left
Epoch [4/720], Loss: 5273818.8000 about 717.0 min   or 11.950 hrs left
Epoch [5/720], Loss: 4670460.4000 about 716.0 min   or 11.933 hrs left
Epoch [6/720], Loss: 3458096.6500 about 715.0 min   or 11.917 hrs left
Epoch [7/720], Loss: 1636167.1000 about 714.0 min   or 11.900 hrs left
Epoch [8/720], Loss: 599256.6000 about 713.0 min   or 11.883 hrs left
Epoch [9/720], Loss: 754331.0500 about 712.0 min   or 11.867 hrs left
Epoch [10/720], Loss: 273213.7875 about 711.0 min   or 11.850 hrs left
Epoch [11/720], Loss: 264518.2094 about 710.0 min   or 11.833 hrs left
Epoch [12/720], Loss: 253025.5125 about 709.0 min   or 11.817 hrs left
Epoch [13/720], Loss: 194490.3625 about 708.0 min   or 11.800 hrs left
Epoch [14/720], Loss: 209805.2875 about 707.0 min   or 11.783 hrs left
Epoch [1

In [6]:
torch.save(model2.state_dict(), './fan_corner2_detector_1129.pth')

# keep training

In [ ]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points2_less100.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

# Training loop
num_epochs = 530
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model2(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # 1 epoch for 1 min (selected_points2_1127)
    # print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min   or %-.3f hrs left'% ((num_epochs-epoch)/60))
    # 1 epoch for 16sec (selected_points2_less100)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/3.9,'min   or %-.3f hrs left'% ((num_epochs-epoch)/3.9/60))

Epoch [1/530], Loss: 27218.1398 about 135.8974358974359 min   or 2.265 hrs left
Epoch [2/530], Loss: 12741.2415 about 135.64102564102564 min   or 2.261 hrs left
Epoch [3/530], Loss: 18784.8882 about 135.3846153846154 min   or 2.256 hrs left
Epoch [4/530], Loss: 8220.7944 about 135.12820512820514 min   or 2.252 hrs left
Epoch [5/530], Loss: 13397.0859 about 134.87179487179486 min   or 2.248 hrs left
Epoch [6/530], Loss: 3568.9695 about 134.6153846153846 min   or 2.244 hrs left
Epoch [7/530], Loss: 8504.6663 about 134.35897435897436 min   or 2.239 hrs left
Epoch [8/530], Loss: 5844.2551 about 134.10256410256412 min   or 2.235 hrs left
Epoch [9/530], Loss: 3001.5054 about 133.84615384615384 min   or 2.231 hrs left
Epoch [10/530], Loss: 5717.3337 about 133.5897435897436 min   or 2.226 hrs left
Epoch [11/530], Loss: 2633.1703 about 133.33333333333334 min   or 2.222 hrs left
Epoch [12/530], Loss: 3002.0641 about 133.07692307692307 min   or 2.218 hrs left
Epoch [13/530], Loss: 3332.8904 about

In [5]:
torch.save(model2.state_dict(), './fan_corner2_detector_1210.pth')